# Imports and set up for SimStore

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import openpathsampling as paths
from tqdm.auto import tqdm
import numpy as np

In [ ]:
from openpathsampling.experimental.storage import monkey_patch_all, Storage

paths = monkey_patch_all(paths)

In [ ]:
from openpathsampling.analysis import tis
from openpathsampling.analysis.tis.core import steps_to_weighted_trajectories

# Load things from storage

In [ ]:
# remember that we stored these in the setup file
# we could have appended these tags to the output, but didn't
setup = Storage("setup.db", mode='r')
tau = setup.tags['tau']
sigma_nm = setup.tags['sigma_nm']
setup.close()

In [ ]:
storage = Storage("tis.db", mode='r')

In [ ]:
scheme = storage.schemes['retis_scheme']
network = storage.networks['tis']

In [ ]:
# we use these to extract info from TIS analysis
condensed = storage.volumes['condensed']
extended = storage.volumes['extended']

# Move summary

Check the acceptance of your movers -- problems here are important to catch quickly.

In [ ]:
scheme.move_summary(storage.steps)

## Acceptance of movers by move group/type

In [ ]:
scheme.move_summary(movers='shooting')

In [ ]:
scheme.move_summary(movers='repex')

In [ ]:
scheme.move_summary(movers='pathreversal')

Here's how you can find the named of those "groups" of movers (this will depend on the move scheme used):

In [ ]:
scheme.movers.keys()

You can also get more details on some movers by passing the mover instance to `move_summary`:

In [ ]:
scheme.move_summary(movers=scheme.movers['shooting'][2])

# Weighted trajectories

TODO: write this

Another object that we can build and then re-use is the `weighted_trajectories`. Over the course of Monte Carlo sampling, different samples accumulate different weights. Since path sampling will simultaneously sample multiple ensembles, we need to gather these weights for each ensemble.

The `steps_to_weighted_trajectories` method calculates the weights for each trajectory in each ensemble. It returns a Python `dict` that maps ensembles to `Counter`s. A `Counter` is like a `dict` that maps a key (in this case a `Trajectory`) to the appropriate weights.

In [ ]:
weighted_trajs = steps_to_weighted_trajectories(tqdm(storage.steps),
                                                ensembles=network.all_ensembles)

# Path length distributions

In [ ]:
fig, axs = plt.subplots(2, 2)
for ax, (ens, counter) in zip(axs.flatten(), weighted_trajs.items()):
    lengths = [len(traj) for traj in counter.keys()]
    weights = list(counter.values())
    
    if isinstance(ens, paths.TISEnsemble):    
        # this is a trick to plot all TIS ensembles with the same bins
        # here, I want 25 bins over the range of 0 to 50 frames
        bins = np.linspace(0, 50, 25)
    else:
        # this is for the minus ensemble
        bins = 25
        
    ax.hist(lengths, bins=bins, weights=weights)
    ax.set_title(ens.name)
    ax.set_xlabel('Path length (frames)')
plt.subplots_adjust(hspace=0.75, wspace=0.3)
plt.close(fig)  # so I only display in the next cell/slide

In [ ]:
fig

# TIS Analysis

This will give us the rates, but also important information about simulation convergence.

In [ ]:
analysis = tis.StandardTISAnalysis(
    network,
    steps=storage.steps,
    scheme=scheme,
    max_lambda_calcs={t: {'bin_width': 0.02 * sigma_nm,
                          'bin_range': (1.16 * sigma_nm, 1.7 * sigma_nm)}
                      for t in network.sampling_transitions}
)

In [ ]:
for ens in network.sampling_ensembles:
    cp = analysis.crossing_probability(ens)
    plt.plot(cp.x / sigma_nm, cp, label=ens.name)
    
tot_cp = analysis.total_crossing_probability[(condensed, extended)]
plt.plot(tot_cp.x / sigma_nm , tot_cp, lw=2, color='k', label="Total crossing probability")
plt.yscale('log')
plt.ylabel("Crossing probability")
plt.xlabel("r ($\sigma$)")
plt.legend(loc="lower left");
fig = plt.gcf()
plt.close(fig)

In [ ]:
fig

# Rates

TODO: write this

## Rate matrix

In [ ]:
rate_matrix = analysis.rate_matrix()
rate_matrix.to_pandas()

In [ ]:
print(rate_matrix.to_pandas().to_latex())

## Individual rates

In [ ]:
rate = rate_matrix[condensed, extended]
print(rate)

In [ ]:
# get this in units of tau^{-1} instead of ps^{-1}
rate_matrix[(condensed, extended)] * tau

# Fluxes

In [ ]:
tis.flux_matrix_pd(analysis.flux_matrix)

In [ ]:
import pandas as pd
df = pd.DataFrame(tis.flux_matrix_pd(analysis.flux_matrix))
df